In [2]:
from sklearn.preprocessing import LabelEncoder
# ^^^ pyforest auto-imports - don't write above this line
import os
import pandas as pd
import csv


def csv_to_dataframes(output='ps'):
    ''' Returns 2 dataframes

    Extracts 1 dataframe with paragraphs and 1 dataframe with
    sentences from a csv file. The csv files names' are parsed
    assuming the following syntax:
    "author_name - title - publication_date.csv"
    '''
    ##########################b#####################
    ###y####  convert csv to df_paragraphs  ########
    ################################################

    # Get csv path ; the csv files are arrays of pre-selected* paragraphs
    # that were extracted from raw txt files by * (cf. Lilou)
    csv_path= "/Users/cyrielle/code/Cyr-dcx/author_style/author_style/data/comp_aut"


    # Create a list of book names
    books = [
        csv_file for csv_file in os.listdir(csv_path)
        if csv_file.endswith('.csv')]


    # Parsing csv file names to get author names, book titles and publishing date
    # and putting these elements in lists that have the same index as the list 'books'
    authors = [csv_file.split(' ')[0]+' '+csv_file.split(' ')[1] for csv_file in books]
    titles = [csv_file.split(' - ')[1] for csv_file in books]
    book_dates = [csv_file.split(' - ')[2].replace('.csv','') for csv_file in books]

    # Initializing a list of dataframes
    dfs = []

    # For each book (in the list 'books'),
    ## 1. create a dataframe with 1 paragraph per row
    ## 2. create columns with fixed values for other features than text
    ## 3. append the dataframe in the list 'dfs' of dataframes
    ## containing the paragraphs from all books

    for book in books:
        ## 1.
        df_temp = pd.read_csv(os.path.join(csv_path,book), header=None)
        ## 2.
        df_temp['author'] = authors[books.index(book)]
        df_temp['title'] = titles[books.index(book)]
        df_temp['book_date'] = book_dates[books.index(book)]
        ## 3.
        dfs.append(df_temp)

    ## Concatenate all dataframes in 'dfs' to get
    ## a single dataframe with paragraphs from all books
    df_paragraphs = pd.concat([df for df in dfs], ignore_index = True, axis=0)
    df_paragraphs.rename(mapper={0:"text"}, axis=1, inplace=True) # NB: The column name for the root_path text is explicitly called in a preprocessing function, it must be 'text'

    ###############y########################################
    ########  convert df_paragraphs to df_sentences  #######
    #######################################b################

    # Initializing a list of dataframes
    dfs = []

    # For each paragraph of our dataset (i.e. for each row in df_paragraph):
    for i in range(df_paragraphs['text'].count()):

        # Separate sentences with '. ' as a delimiter
        # (careful: "J. C.", "Mr.", [...]) ignore ?
        sentences = str(df_paragraphs.text[i]).split(". ")

        # Prepare columns with fixed values for Author_name, Title and Book_date,
        # to assign each sentence of a paragraph to the same Author_name, Title and Book_date.
        author_temp = [df_paragraphs.author[i] for k in range(len(sentences))]
        title_temp = [df_paragraphs.title[i] for k in range(len(sentences))]
        date_temp = [df_paragraphs.book_date[i] for k in range(len(sentences))]

        # Concatenate the 4 previous lists to build a single dataframe
        # containing all sentences of the i-th paragraph of df_paragraphs
        data = [sentences, author_temp, title_temp, date_temp]
        df_temp = pd.DataFrame(data).T

        # Build the list of dataframes containing all sentences of our dataset
        dfs.append(df_temp)

    # Assemble the dataframe containing all sentences of our dataset
    df_sentences = pd.concat(dfs, ignore_index = True, axis=0)
    df_sentences.rename(mapper={0:"text", 1: 'author', 2:'title', 3 : 'book_date'}, axis=1, inplace=True)

    if output == 'p':
        return df_paragraphs
    if output == 's':
        return df_sentences
    if output == 'ps':
        return df_paragraphs, df_sentences

In [1]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import unidecode
#import spacy
import numpy as np
import pandas as pd

#nlp = spacy.load("fr_core_news_sm")


def preprocess(text,
               punctuation=False,
               lower_case=True,
               remove_stopwords=False,
               accents=True,
               numbers=True,
               lemmatize=False,
               language='french'):

    if numbers:
        text = ''.join(char for char in text if not char.isdigit())
    if punctuation:
        text = ''.join(char for char in text if not char in string.punctuation)
    if lower_case:
        text = text.lower()
    if accents:
        text = unidecode.unidecode(text)
    if remove_stopwords:
        stop_words = set(stopwords.words(language))
        word_tokens = word_tokenize(text)
        text = ' '.join(char for char in word_tokens if not char in stop_words)
    if lemmatize:
        text = word_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        lemmatized = [lemmatizer.lemmatize(char) for char in text]
        text = ' '.join(lemmatized)
    return text


def add_cleaned_column(df):
    df["preprocess_data"] = df['text'].apply(lambda x: preprocess(x))
    return df


"""def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]"""


"""def return_word_embedding(sentence):
    # Vectoriser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]"""


def stopword_count(text):
    stop_words = set(stopwords.words('french'))
    word_tokens = word_tokenize(text)
    stopword_count = len([w for w in word_tokens if w in stop_words])
    return stopword_count


def vocab_richness(text):
    tokens = word_tokenize(text)
    total_length = len(tokens)
    unique_words = set(tokens)
    unique_word_length = len(unique_words)
    if total_length > 0:
        return unique_word_length / total_length
    else:
        return 0

def sentence_count(x):
    if len(x.split()) >0:
        return x.count('.') / len(x.split())
    else:
        return 0


def features(df, output='p'):
    if output=='p':

        df['preprocess_data'] = df['text'].apply(lambda x: preprocess(x))
        df['word_ratio'] = df['text'].apply(lambda x: len(x.split()))
        df['unique_word'] = df['text'].apply(
        lambda x: 0 if len(x.split())==0 else (len(np.unique(x.split()))/ len(x.split())))

        df['sentences_ratio'] = df['text'].apply(lambda x: 0 if len(x.split())==0 else x.count('.') / len(x.split()))
        df['stopwords_ratio'] = df['text'].apply(lambda x: 0 if len(x.split(
    )) == 0 else (stopword_count(x) / len(x.split())))
        df['vocab richness'] = df['text'].apply(vocab_richness)
        return df

    elif output=='s':
        df['preprocess_data'] = df['text'].apply(lambda x: preprocess(x))
        df['word_ratio'] = df['text'].apply(lambda x: len(x.split()))
        df['unique_word_ratio'] = df['text'].apply(
        lambda x: 0 if len(x.split())==0 else (len(np.unique(x.split()))/ len(x.split())))

        df['stopwords_ratio'] = df['text'].apply(lambda x: 0 if len(x.split(
    )) == 0 else (stopword_count(x) / len(x.split())))
        df['vocab richness'] = df['text'].apply(vocab_richness)
        return df


## Créer le dataFrame

In [ ]:
df = csv_to_dataframes(output="p")

## Appliquer le preprocessing

In [ ]:
df = features(df, output='p')

In [ ]:
df.head()

## Créer X et y

In [ ]:
#selection de X et y dans le dataframe df
X = df[['preprocess_data']]
y = df["author"]

In [9]:
X

,preprocess_data
0,pendant que nous franchissions la porte du nor...
1,"<< en apres jesus-christ, les troupes de syag..."
2,van eyck presenta la vierge au chanoine van de...
3,"un an apres l'insolence du soldat, clovis rass..."
4,les hommes se font une idee grotesque du temps...
...,...
17774,c'est a bord d'un train de la southern pacific...
17775,"quelle que soit, pour signer, la solution adop..."
17776,reste la possibilite d'aller faire un tour dan...
17777,"le lendemain matin, il se leve tard, trainant ..."


In [7]:
# Label Encode y
cat_transformer = LabelEncoder()
y = cat_transformer.fit_transform(y)

<IPython.core.display.Javascript object>

## Sauvegarder X et y sur le package


In [30]:
import joblib

joblib.dump(X,"X.pkl")


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [31]:
joblib.dump(y,"y.pkl")

AttributeError: 'numpy.ndarray' object has no attribute 'values'